In [1]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger

# import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "max_split_size_mb:128"
import torch
import numpy as np
import pandas as pd

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

c:\Users\sonng\anaconda3\envs\tsff_env\lib\site-packages\pytorch_forecasting\models\base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [22]:
from ESRNN.m4_data import *
from ESRNN.utils_evaluation import evaluate_prediction_owa, smape, mase, evaluate_panel
from ESRNN.utils_visualization import plot_grid_prediction

In [3]:
X_train_df, y_train_df, X_test_df, y_test_df = prepare_m4_data(dataset_name="Hourly",
                                                               directory="../../data/M4",
                                                               num_obs=414)

In [4]:
unique_ids = y_train_df['unique_id'].unique()

In [5]:
all_forecasts = {}

In [6]:
pl.seed_everything(42)

Seed set to 42


42

In [7]:
df_arima_train = pd.read_csv('../../results/m4/base_model_train_set/y_hat_df_arima_ts.csv')
df_theta_train = pd.read_csv('../../results/m4/base_model_train_set/y_hat_df_theta_ts.csv')
df_xgb_train = pd.read_csv('../../results/m4/base_model_train_set/y_hat_df_xgb_ts.csv')
df_gru_train = pd.read_csv('../../results/m4/base_model_train_set/y_hat_df_gru_ts.csv')
df_lstm_train = pd.read_csv('../../results/m4/base_model_train_set/y_hat_df_lstm_ts.csv')

In [8]:
df_arima_test = pd.read_csv('../../results/m4/y_hat_df_arima.csv')
df_theta_test = pd.read_csv('../../results/m4/y_hat_df_theta.csv')
df_xgb_test = pd.read_csv('../../results/m4/y_hat_df_xgb.csv')
df_gru_test = pd.read_csv('../../results/m4/y_hat_df_gru.csv')
df_lstm_test = pd.read_csv('../../results/m4/y_hat_df_lstm.csv')

### Creating the base model prediction data frame

#### Train set

In [ ]:
base_models = ['arima', 'theta', 'xgboost', 'gru', 'lstm']
data_frames = [df_arima_train, df_theta_train, df_xgb_train, df_gru_train, df_lstm_train]

In [48]:
df_base_models = pd.DataFrame()

In [49]:
df_base_models[['unique_id', 'ds']] = df_arima_train[['unique_id', 'ds']]

In [50]:
df_base_models

,unique_id,ds
0,H1,1970-01-23 04:00:00
1,H1,1970-01-23 05:00:00
2,H1,1970-01-23 06:00:00
3,H1,1970-01-23 07:00:00
4,H1,1970-01-23 08:00:00
...,...,...
69547,H99,1970-01-29 23:00:00
69548,H99,1970-01-30 00:00:00
69549,H99,1970-01-30 01:00:00
69550,H99,1970-01-30 02:00:00


In [51]:
df_base_models = pd.concat([df_base_models] + [df[['y_hat']].rename(columns={'y_hat' : model}) for df, model in zip(data_frames, base_models)], axis=1)

In [53]:
df_base_models['y'] = y_train_df.groupby('unique_id').tail(168)['y'].reset_index(drop=True)
df_base_models

,unique_id,ds,arima,theta,xgboost,gru,lstm,y
0,H1,1970-01-23 04:00:00,640.78800,638.21857,797.35205,703.12354,700.60535,635.0
1,H1,1970-01-23 05:00:00,573.37900,572.25050,749.65130,635.24634,634.41370,577.0
2,H1,1970-01-23 06:00:00,527.94617,524.86346,743.40480,573.02810,569.60670,533.0
3,H1,1970-01-23 07:00:00,492.67783,490.73680,685.00370,525.73480,524.29940,504.0
4,H1,1970-01-23 08:00:00,468.57578,468.91650,622.73470,493.56607,493.81927,485.0
...,...,...,...,...,...,...,...,...
69547,H99,1970-01-29 23:00:00,22628.06800,20837.17200,21028.18000,20118.81800,24763.27300,27926.0
69548,H99,1970-01-30 00:00:00,21908.38700,20241.95900,21035.61300,18960.70900,24508.59400,26744.0
69549,H99,1970-01-30 01:00:00,21146.44300,19872.22000,21088.26600,17356.62000,23856.59800,25829.0
69550,H99,1970-01-30 02:00:00,20488.85500,19540.13500,21595.63300,15822.04800,23196.18200,25421.0


#### Test set

In [108]:
base_models = ['arima', 'theta', 'xgboost', 'gru', 'lstm']
data_frames = [df_arima_test, df_theta_test, df_xgb_test, df_gru_test, df_lstm_test]

In [109]:
df_base_models_test = pd.DataFrame()

In [110]:
df_base_models_test[['unique_id', 'ds']] = df_arima_test[['unique_id', 'ds']]

In [111]:
df_base_models_test

,unique_id,ds
0,H1,1970-01-30 04:00:00
1,H1,1970-01-30 05:00:00
2,H1,1970-01-30 06:00:00
3,H1,1970-01-30 07:00:00
4,H1,1970-01-30 08:00:00
...,...,...
19867,H99,1970-01-31 23:00:00
19868,H99,1970-02-01 00:00:00
19869,H99,1970-02-01 01:00:00
19870,H99,1970-02-01 02:00:00


In [112]:
df_base_models_test = pd.concat([df_base_models_test] + [df[['y_hat']].rename(columns={'y_hat' : model}) for df, model in zip(data_frames, base_models)], axis=1)

In [113]:
df_base_models_test['y'] = y_test_df.groupby('unique_id').tail(168)['y'].reset_index(drop=True)
df_base_models_test

,unique_id,ds,arima,theta,xgboost,gru,lstm,y
0,H1,1970-01-30 04:00:00,614.17645,620.21350,755.18730,687.63950,685.46387,619.0
1,H1,1970-01-30 05:00:00,541.41406,555.38140,725.16595,626.73047,623.76215,565.0
2,H1,1970-01-30 06:00:00,504.21580,510.38382,703.59150,565.96844,561.43317,532.0
3,H1,1970-01-30 07:00:00,475.11530,477.62396,650.84937,523.33150,517.42017,495.0
4,H1,1970-01-30 08:00:00,453.99097,456.31180,601.89606,491.06958,485.30426,481.0
...,...,...,...,...,...,...,...,...
19867,H99,1970-01-31 23:00:00,27710.54500,26598.95000,21476.23000,29159.08000,29104.52500,24039.0
19868,H99,1970-02-01 00:00:00,26604.14300,25796.86500,22039.80300,28519.88000,28023.54300,22946.0
19869,H99,1970-02-01 01:00:00,25663.48400,25299.76800,22500.24800,27937.31200,26967.72900,22217.0
19870,H99,1970-02-01 02:00:00,25152.79500,24848.34800,22126.19300,27420.53100,26249.66800,21416.0


In [115]:
df_base_models_test.to_csv('../../results/m4/base_model_test_set/df_base_models.csv', index=False)

### Proceeding to create base models error contribution data frame

In [70]:
y_panel = df_base_models[['unique_id', 'ds', 'y']]
y_hat_panel_fun = lambda model_name: df_base_models[['unique_id', 'ds', model_name]].rename(columns={model_name: 'y_hat'})

In [71]:
model_names = set(df_base_models.columns) - set(y_panel.columns)
model_names

{'arima', 'gru', 'lstm', 'theta', 'xgboost'}

In [72]:
errors_smape = y_panel[['unique_id']].drop_duplicates().reset_index(drop=True)
errors_mase = errors_smape.copy()


In [73]:
y_insample_df = y_train_df.groupby('unique_id').apply(lambda x: x.iloc[:-168]).reset_index(drop=True)

In [84]:
for model_name in model_names:
    errors_smape[model_name] = None
    errors_mase[model_name] = None
    y_hat_panel = y_hat_panel_fun(model_name)

    errors_smape[model_name] = evaluate_panel(y_panel, y_hat_panel, smape)
    errors_mase[model_name] = evaluate_panel(y_panel, y_hat_panel, mase, y_insample_df, seasonality=24)

In [85]:
errors_smape

,lstm,gru,arima,theta,xgboost
unique_id,,,,,
H1,0.071051,0.080442,0.052096,0.050742,0.214728
H10,0.048537,0.041891,0.044421,0.028533,0.069098
H100,0.310653,0.430661,0.193501,0.166960,0.329247
H101,0.132635,0.142992,0.066542,0.068664,0.084066
H102,0.198840,0.216083,0.149146,0.155246,0.272766
...,...,...,...,...,...
H95,0.058830,0.056479,0.060139,0.064421,0.157611
H96,0.074631,0.099854,0.127831,0.082887,0.197288
H97,0.144434,0.028256,0.030777,0.029238,0.121188


In [86]:
errors_mase

,lstm,gru,arima,theta,xgboost
unique_id,,,,,
H1,1.015330,1.146595,0.748045,0.725544,3.035923
H10,2.027233,1.748494,1.883617,1.198519,2.899977
H100,2.301098,3.030150,1.378321,1.119803,2.269879
H101,1.370131,1.526816,0.704732,0.727772,0.894813
H102,1.428595,1.466951,1.141811,1.174502,2.022558
...,...,...,...,...,...
H95,1.211895,1.149397,1.265043,1.356891,3.159726
H96,0.990770,1.304006,1.651226,1.069987,2.647422
H97,3.839082,0.761165,0.824287,0.783427,3.169866


In [87]:
mean_smape_benchmark = errors_smape[base_models].mean()
mean_mase_benchmark = errors_mase[base_models].mean()

In [92]:
mean_smape_benchmark

arima      0.153688
theta      0.185320
xgboost    0.239229
gru        0.267754
lstm       0.229306
dtype: float64

In [93]:
mean_mase_benchmark

arima      1.572499
theta      2.521568
xgboost    4.295293
gru        5.881229
lstm       4.738043
dtype: float64

In [96]:
errors = errors_smape/mean_smape_benchmark + errors_mase/mean_mase_benchmark
errors = 0.5*errors
errors = errors

In [98]:
errors

,arima,gru,lstm,theta,xgboost
unique_id,,,,,
H1,0.407339,0.247695,0.262074,0.280770,0.802193
H10,0.743442,0.226877,0.319766,0.314637,0.481994
H100,1.067783,1.061824,0.920210,0.672510,0.952370
H101,0.440565,0.396826,0.433799,0.329567,0.279864
H102,0.848281,0.528226,0.584328,0.651752,0.805533
...,...,...,...,...,...
H95,0.597894,0.203186,0.256168,0.442867,0.697226
H96,0.940912,0.297327,0.267287,0.435798,0.720518
H97,0.362223,0.117476,0.720071,0.234230,0.622281


### Proceeding to train the FFORMA 